In [38]:
import numpy as np
from scipy.stats import poisson, norm, multivariate_normal
from numpy.linalg import inv
from numpy.linalg import multi_dot

In [225]:
np.diagflat(np.array([2.0,4.0])**-1)

array([[0.5 , 0.  ],
       [0.  , 0.25]])

# General funtions

In [390]:
def pois_ll(X, y, beta):
    Xb = np.dot(X,beta)
#     print(Xb)
    ll = np.dot(Xb,y) - np.sum(np.exp(Xb))
#     print('np.sum(np.exp(Xb))')
#     print(np.sum(np.exp(Xb)))
#     print('np.dot(Xb,y)')
#     print(np.dot(Xb,y))
#     print(ll)
    return ll

def nb_ll(X, y, beta, alpha):
    Xb = np.dot(X,beta)
#     print('np.dot(np.ones(y.shape[0]) * alpha + y , np.log(np.exp(Xb) + alpha))')
#     print(np.dot(np.ones(y.shape[0]) * alpha + y , np.log(np.exp(Xb) + alpha)))
#     print('np.dot(Xb,y)')
#     print(np.dot(Xb,y))
    ll = np.dot(y,Xb) - np.dot(np.ones(y.shape[0]) * alpha + y , np.log(np.exp(Xb) + alpha))
    return np.sum(ll)

# IRLS and Forward Selection

In [ ]:
def IRLS(X, y, reg_type: ['poisson','nb']
         , alpha = None, threshold = 0.01
         , just_score = True):
    # Distribution specifics
#     beta = np.zeros((X.shape[1],1))
    beta = np.zeros((X.shape[1]))
    if reg_type == 'poisson':
        ll_cur = pois_ll(X, y, beta)
        W = np.diagflat(np.exp(np.dot(X,beta)))
#         print(W)
        D = np.diagflat(np.exp(np.dot(X,beta)))
#         print(D)
        z = np.dot(X,beta) + np.dot(inv(D),(y-np.exp(np.dot(X,beta))))
#        
        beta_est = multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W, z])
#         print('beta est')
#         print(beta_est)
        ll_next = pois_ll(X, y, beta_est)
#         print(ll_cur)
#         print(ll_next)
#         print(' ll_next - ll_cur:{}'.format(ll_next - ll_cur))
        while ll_next - ll_cur > threshold:
#             print('while loop')
            ll_cur = pois_ll(X, y, beta_est)
#             print(ll_cur)
            W = np.diagflat(np.exp(np.dot(X,beta_est)))
#             print(W)
            D = np.diagflat(np.exp(np.dot(X,beta_est)))
#             print(D)
            z = np.dot(X,beta_est) + np.dot(inv(D),(y-np.exp(np.dot(X,beta_est))))
#             print('z')
#             print(z)
            beta_est = multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W, z])
#             print('beta_est')
#             print(beta_est)
#             beta_est = np.dot(multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W]), z)
#             print(beta_est)
            ll_next = pois_ll(X, y, beta_est)
#             print(ll_next)
#             print(' ll_next - ll_cur:{}'.format(np.round(ll_next - ll_cur,3)))
#             print(' ll_next:{}'.format(ll_next))
#             print(' ll_cur:{}'.format(ll_cur))
    
    if reg_type == 'nb':
        ll_cur = nb_ll(X, y, beta, alpha)
        W = np.diagflat(np.exp(np.dot(X,beta))/(1+(alpha**-1) * np.exp(np.dot(X,beta)))) # adjust to nb
        D = np.diagflat(np.exp(np.dot(X,beta)))
        z = np.dot(X,beta) + np.dot(inv(D),(y-np.exp(np.dot(X,beta))))
        beta_est = np.dot(multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W]), z)
#         print('beta_est')
#         print(beta_est)
        ll_next = nb_ll(X, y, beta_est, alpha)
    # Iterative process
        while ll_next - ll_cur > threshold:
#             print('while loop')
            ll_cur = nb_ll(X, y, beta_est, alpha)
            W = np.diagflat(np.exp(np.dot(X,beta_est))/(1+(alpha**-1) * np.exp(np.dot(X,beta_est)))) # adjust to nb
            D = np.diagflat((np.exp(np.dot(X,beta_est))))
            z = np.dot(X,beta_est) + np.dot(inv(D),(y-np.exp(np.dot(X,beta_est))))
            beta_est = np.dot(multi_dot([inv(multi_dot([np.transpose(X),W,X])) , np.transpose(X), W]), z)
            ll_next = nb_ll(X, y, beta_est, alpha)
#             print(ll_next)
#             print(' ll_next - ll_cur:{}'.format(np.round(ll_next - ll_cur,3)))
#             print(' ll_next:{}'.format(ll_next))
#             print(' ll_cur:{}'.format(ll_cur))
    
    if just_score:
        return ll_next
    else:
        return beta_est, ll_next, np.exp(np.dot(X,beta_est))

def fwd(X,y,reg_type: ['poisson','nb'], criteria: ['AIC','BIC','RIC']
        , sel_feat = None, sel_dict = None, alpha = None):
    if sel_feat is None:
        sel_feat = []
    if sel_dict is None:
        sel_dict = {'features':[],'score': np.inf}
#         feat_score =
    for feature in [i for i in range(X.shape[1]) if i not in sel_feat]:
        print(sel_feat)
        print(feature)
        in_feat = sel_feat+[feature]
        print(in_feat)
        print('currently testing:')
        print(in_feat)
        # Criteria definition:
        if criteria == 'AIC':
            pen = 2* len(in_feat)
        if criteria == 'BIC':
            pen = np.log(X.shape[0])* len(in_feat)
        if criteria == 'RIC':
            pen = 2* np.log(X.shape[1])* len(in_feat)
        feat_score_next = -IRLS(X[:,in_feat],y,reg_type,alpha)
        if feat_score_next < sel_dict['score']:
            feat_score = feat_score_next + pen # taking the ll score
            sel_dict['features'] = in_feat
            sel_dict['score'] = feat_score
            sel_dict['-ll']  = feat_score_next
    return sel_dict

In [ ]:
criteria = 'AIC'
score_start = np.inf
sel_dict, sel_feat = fwd(X,y,'poisson', criteria)
print(sel_dict, sel_feat)
counter = 0

In [352]:
for i in range(X.shape[1]):
    print(X[i])

[1.34970702 1.19392043 0.21052964]
[-0.03645668  0.93501969  0.21038773]
[1.13848497 0.75873356 0.83173875]


In [505]:
from numpy.linalg import norm
# np.random.seed(5)
mean = np.zeros(5)
cov = np.diagflat(np.ones(5))
X = np.random.multivariate_normal(mean, cov, size = 2000)
X = X/norm(X, axis = 0)
beta = np.random.rand(5)
y = poisson.rvs(np.exp(np.dot(X,beta)))

In [507]:
X, beta,y

(array([[-0.00292034,  0.0304289 , -0.01685068,  0.00176003,  0.0159383 ],
        [-0.01531048, -0.00238966, -0.02474197,  0.01569092,  0.00462085],
        [ 0.02031691, -0.00247595,  0.0208372 ,  0.02266214, -0.07863014],
        ...,
        [ 0.00190756,  0.01808011,  0.01320618,  0.00331982,  0.02433902],
        [-0.01930915, -0.01902859,  0.00075008,  0.00287682, -0.02035189],
        [-0.00460718,  0.04534486,  0.02166775,  0.01382425,  0.02016516]]),
 array([0.91670318, 0.14661434, 0.64156933, 0.63052196, 0.80866833]),
 array([0, 0, 3, ..., 0, 1, 2]))

In [497]:
# norm(X/norm(X,axis = 0),axis =0)

array([1., 1., 1., 1., 1.])

In [416]:
counter = 0
score_start = np.inf
final_scores = {'AIC': {}, 'BIC': {} ,'RIC': {}}
for crit in ['AIC','BIC','RIC']:
    print('using penalty:{}'.format(crit))
#     criteria = crit
    score_start = np.inf
    sel_dict = fwd(X,y,'poisson', crit)
    print(sel_dict)
    counter = 0
    while score_start > sel_dict['score']:
        counter +=1
        score_start = sel_dict['score']
        sel_dict = fwd(X,y,'poisson',crit, sel_feat = sel_dict['features'], sel_dict = sel_dict)
        print('finished')
        print(counter)
    print('Results')
    print(sel_dict)
    final_scores[crit] = sel_dict

using penalty:AIC
[]
0
[0]
currently testing:
[0]
 ll_next - ll_cur:26.188653918102546
while loop
-1973.8021318521758
 ll_next - ll_cur:0.009
 ll_next:-1973.8021318521758
 ll_cur:-1973.8113460818975
[]
1
[1]
currently testing:
[1]
 ll_next - ll_cur:68.30698350470766
while loop
-1931.6651416570776
 ll_next - ll_cur:0.028
 ll_next:-1931.6651416570776
 ll_cur:-1931.6930164952923
while loop
-1931.6651415846038
 ll_next - ll_cur:0.0
 ll_next:-1931.6651415846038
 ll_cur:-1931.6651416570776
[]
2
[2]
currently testing:
[2]
 ll_next - ll_cur:12.10017017600535
while loop
-1987.8991527455787
 ll_next - ll_cur:0.001
 ll_next:-1987.8991527455787
 ll_cur:-1987.8998298239947
{'features': [1], 'score': 1933.6651415846038, '-ll': 1931.6651415846038}
[1]
0
[1, 0]
currently testing:
[1, 0]
 ll_next - ll_cur:94.01190954402273
while loop
-1905.8863171900484
 ll_next - ll_cur:0.102
 ll_next:-1905.8863171900484
 ll_cur:-1905.9880904559773
while loop
-1905.8863157149099
 ll_next - ll_cur:0.0
 ll_next:-1905.88

In [417]:
final_scores

{'AIC': {'features': [1, 0, 2],
  'score': 1900.8151242215831,
  '-ll': 1894.8151242215831},
 'BIC': {'features': [1, 2, 0],
  'score': 1917.6178316002095,
  '-ll': 1894.8151242215831},
 'RIC': {'features': [1, 0, 2],
  'score': 1901.4067979535919,
  '-ll': 1894.8151242215831}}

In [581]:
(np.ones(5)+5)/(np.ones(5)+1)

array([3., 3., 3., 3., 3.])

# FISTA

In [603]:
from numpy import linalg as LA

def pois_nll_grad(X,y,beta):
    Xb = np.dot(X,beta)
    nll_grad = np.dot(np.transpose(X),np.exp(Xb) - y)
    return nll_grad

def nb_nll_grad(X,y,beta,alpha):
    Xb = np.dot(X,beta)
#     print('np.exp(Xb) + alpha')
#     print(np.exp(Xb) + alpha)
    sec_factor = (np.exp(Xb) - y)/(np.exp(Xb) + alpha)
#     print('sec_factor')
#     print(sec_factor)
#     print('np.dot(np.transpose(X),sec_factor)')
#     print(np.dot(np.transpose(X),sec_factor))
    nll_grad = alpha * np.dot(np.transpose(X),sec_factor)
#     print
    return nll_grad

def L_pois(X,y):
    eigs = LA.eigh(np.dot(np.transpose(X),X))[0]
    idx = eigs.argsort()[::-1][0]  
    eig_max = eigs[idx]
    return np.mean(y) * eig_max
    
def L_nb(X,y,alpha):
    eigs = LA.eigh(np.dot(np.transpose(X),X))[0]
    idx = eigs.argsort()[::-1][0]  
    eig_max = eigs[idx]
    return (alpha + np.mean(y))/alpha * eig_max

def prox(grad, beta, L, pen_vec):
    prox_inp = beta - 1/L * grad
    prox_out = np.maximum(np.abs(prox_inp) - pen_vec,0) * np.sign(prox_inp)
    return prox_out

def FISTA(X, y, pen_vec, 
          type: ['poisson', 'nb'],
          iterations = 4000, is_ordered = True
          ,alpha = None
          ):
    """
    X - Design matrix
    y - dependent variables
    """
    beta_start = np.zeros(X.shape[1])
    w_start = np.zeros(X.shape[1])
    delta_start = 1
    for k in range(iterations):
        if type == 'poisson':
            grad = pois_nll_grad(X, y, beta_start)
            L = L_pois(X,y)
        elif type == 'nb':
            grad = nb_nll_grad(X, y, beta_start,alpha)
            L = L_nb(X,y,alpha)
        ### Ordering for the thresholding ###
        
        if is_ordered:

            indx = np.argsort(beta_start)[::-1]

            beta_start = beta_start[indx]
            ind_dict = {i: j for i,j in zip([*range(beta_start.shape[0])], indx)}
        ### Starting the FISTA. Pay attention that we must sort grad according to indx
        
        w_next = prox(grad[indx], beta_start, L, pen_vec)
        delta_next = (1 + np.sqrt(1 + 4*delta_start**2))/2
        beta_next = w_next + ((delta_start - 1)/delta_next)*(w_next - w_start)
        
        ### Re-ordering again for calculating the gradient
        
        beta_start = np.zeros(beta_next.shape[0])
        for origin_ind in ind_dict:
            beta_start[origin_ind] = beta_next[ind_dict[origin_ind]]
        delta_start = delta_next
        w_start = w_next
    
        if k % 500 == 0:
            print(k)
            print('beta_start: {}'.format(beta_start))
            print('beta_next: {}'.format(beta_next))
            print('nll :{}'.format(pois_ll(X,y,beta_next)))
    return beta_next, ind_dict

In [471]:
[*range(5)]

[0, 1, 2, 3, 4]

In [518]:
10*0.05

0.5

In [604]:
pen_vec = np.array(np.array([0.05,0.04,0.03,0.02,0.01]))
# L_pois_t = L_pois(X,y)
print('Poisson')
FISTA_RES_pois = FISTA(X, y, pen_vec, type = 'poisson', iterations = 10000)
print('\n')

Poisson
0
beta_start: [-0.04298505 -1.66198269  2.97036148  1.61806499 -1.03195716]
beta_next: [-1.03195716  1.61806499  2.97036148 -1.66198269 -0.04298505]
nll :-2003.5873011797898
500
beta_start: [-2.45239339  0.          7.39287085 -3.29489391  0.0297336 ]
beta_next: [-3.29489391  0.0297336   7.39287085  0.         -2.45239339]
nll :-2009.3212194031753
1000
beta_start: [-2.45437076  0.          7.40705784 -3.30280371  0.02449409]
beta_next: [-3.30280371  0.02449409  7.40705784  0.         -2.45437076]
nll :-2009.4041653361855
1500
beta_start: [-2.45503386  0.          7.41181955 -3.30545934  0.02273508]
beta_next: [-3.30545934  0.02273508  7.41181955  0.         -2.45503386]
nll :-2009.432078971465
2000
beta_start: [-2.45536621  0.          7.41420689 -3.30679092  0.02185311]
beta_next: [-3.30679092  0.02185311  7.41420689  0.         -2.45536621]
nll :-2009.446087733334
2500
beta_start: [-2.45556588  0.          7.41564143 -3.3075911   0.02132311]
beta_next: [-3.3075911   0.0213231

In [605]:
print('NB')
FISTA_RES_nb = FISTA(X, y, pen_vec, type = 'nb', iterations = 10000, alpha = 1)

NB
0
beta_start: [-0.00331544 -0.40269174  0.72400778  0.37668109 -0.22190194]
beta_next: [-0.22190194  0.37668109  0.72400778 -0.40269174 -0.00331544]
nll :-1999.4450627059787
500
beta_start: [ 1.32447294 -1.71911885  2.37093368 -2.85970621  2.42605606]
beta_next: [ 1.32447294  2.42605606  2.37093368 -1.71911885 -2.85970621]
nll :-2009.1557878974215
1000
beta_start: [-1.5330563   0.87393506  2.29760392 -0.18046041  0.11269584]
beta_next: [ 0.87393506  0.11269584  2.29760392 -0.18046041 -1.5330563 ]
nll :-1995.5726079754334
1500
beta_start: [-0.70308914 -0.18357483  2.15702964  3.06553715 -2.43035867]
beta_next: [ 2.15702964  3.06553715 -0.70308914 -0.18357483 -2.43035867]
nll :-2015.7145145580203
2000
beta_start: [ 1.5996493  -0.56716365  1.84781462  1.28398593 -2.61402414]
beta_next: [ 1.5996493  -0.56716365  1.28398593  1.84781462 -2.61402414]
nll :-1995.6065725634603
2500
beta_start: [ 1.09175103  0.5102909  -0.83344756 -2.35495205  2.97306577]
beta_next: [ 1.09175103  2.97306577  

In [597]:
print(8.78544742e+026)

8.78544742e+26


In [606]:
be = FISTA_RES_pois[0]
ind = FISTA_RES_pois[1]
new_beta = np.zeros(be.shape[0])
for origin_ind in ind:
    new_beta[origin_ind] = be[ind[origin_ind]]
argsort = np.argsort(np.exp(np.dot(X,new_beta)))[::-1]
np.exp(np.dot(X,new_beta))[argsort]
# new_beta

array([1.99771823, 1.96798843, 1.89186073, ..., 0.50643354, 0.48752114,
       0.43884398])

In [607]:
be = FISTA_RES_nb[0]
ind = FISTA_RES_nb[1]
new_beta = np.zeros(be.shape[0])
for origin_ind in ind:
    new_beta[origin_ind] = be[ind[origin_ind]]
argsort = np.argsort(np.exp(np.dot(X,new_beta)))[::-1]
np.exp(np.dot(X,new_beta))[argsort]
# new_beta

array([1.52487127, 1.47809048, 1.41002458, ..., 0.73260225, 0.69481071,
       0.67953378])

# Simulations

In [ ]:
rho = [0,0.5,0.8]
d = [20, 100, 200, 500, 1000]
per = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.5, 0.7, 0.9]